### Lab3

In [1]:
import cv2 as cv
import numpy as np

def to_screen(name, file):
    cv.imshow(name, file)  
    cv.waitKey(0)
    cv.destroyAllWindows()  

### exercise 1

In [58]:
cx = 960
cy = 540
# f = 960
f = 1920

matrix = np.array([[f, 0, cx], [0, f, cy], [0, 0, 1]])

### assignment 1

In [59]:
vertices = np.array([
    [0, 0, 0], 
    [1, 0, 0], 
    [1, 1, 0],  
    [0, 1, 0],  
    [0, 0, 1],  
    [1, 0, 1],  
    [1, 1, 1],  
    [0, 1, 1]   
])

vertices[:,2] += 5
vertices[:,0] += 2

edges = np.array([    [0, 1], [1, 2], [2, 3], [3, 0],  
    [4, 5], [5, 6], [6, 7], [7, 4],  
    [0, 4], [1, 5], [2, 6], [3, 7]   
])


In [60]:
homogeneous_coords = vertices @ matrix.T


image_coords = homogeneous_coords[:, :2] / homogeneous_coords[:, 2:]
image_coords = np.round(image_coords).astype(int)
# image_coords

In [61]:


# create empty 1080p image
image = np.zeros((1080, 1920, 3), dtype=np.uint8)

# draw vertices as green circles
for i in range(8):
    cv.circle(image, tuple(image_coords[i]), 10, (0, 255, 0), -1)

# draw edges as red lines
for i, j in edges:
    cv.line(image, tuple(image_coords[i]), tuple(image_coords[j]), (0, 0, 255), 2)

to_screen("cube", image)


### Question 2: what happens if you double the focal length?

### Question 3

The reason why only even powers of r are included in this polynomial is because of symmetry. The radial distortion model assumes that the distortion is symmetric with respect to the principal point. 

Therefore, any odd power of r would introduce an asymmetry in the distortion model that does not exist in reality.

### Assignment 2

In [62]:
### source: https://learnopencv.com/camera-calibration-using-opencv/

import glob

pattern_size = (10, 6)

objp = np.zeros((pattern_size[0]*pattern_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)

objpoints = []
imgpoints = []

calibration_files = glob.glob("../calibration_frames/*.png")

for fname in calibration_files:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    ret, corners = cv.findChessboardCorners(gray, pattern_size, None)

    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        
        cv.drawChessboardCorners(img, pattern_size, corners, ret)
        cv.imshow('img', img)
        cv.waitKey(500)

cv.destroyAllWindows()




In [63]:
# Use the object and image points to estimate camera parameters
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# Print the camera matrix and distortion coefficients
print("Intrinsic matrix:")
print(mtx)
print("Distortion coefficients:")
print(dist)

Intrinsic matrix:
[[583.72423649   0.         647.13947331]
 [  0.         588.20496961 338.38519019]
 [  0.           0.           1.        ]]
Distortion coefficients:
[[-2.41188339e-01  6.80055862e-02 -8.82713982e-04 -2.37905310e-05
  -9.40313880e-03]]


### Assignment 3 

In [64]:
from numpy import random
objpoints = np.array(objpoints)
imgpoints = np.array(imgpoints)

ret_arr = []
mtx_arr = []
dist_arr = []
rvecs_arr = []
tvecs_arr = []

for i in range(10):
    x = random.randint(len(objpoints), size=(20))
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints[x], imgpoints[x], gray.shape[::-1], None, None)
    ret_arr.append( ret)
    mtx_arr.append( mtx)
    dist_arr.append(dist)
    rvecs_arr.append(rvecs)
    tvecs_arr.append(tvecs)

533.7214764605827

In [66]:
# Extract the parameters cx, cy, f1, and f2 from each camera matrix
cx_list = [matrix[0, 2] for matrix in mtx_arr]
cy_list = [matrix[1, 2] for matrix in mtx_arr]
f1_list = [matrix[0, 0] for matrix in mtx_arr]
f2_list = [matrix[1, 1] for matrix in mtx_arr]

# Calculate the standard deviation of each parameter
cx_std = np.std(cx_list)
cy_std = np.std(cy_list)
f1_std = np.std(f1_list)
f2_std = np.std(f2_list)

print(f"standard deviation across cx {cx_std}")
print(f"standard deviation across cy {cy_std}")
print(f"standard deviation across f1 {f1_std}")
print(f"standard deviation across f2 {f2_std}")


standard deviation across cx 25.63123709911693
standard deviation across cy 19.689454901497086
standard deviation across f1 1289.9989956923607
standard deviation across f2 601.1873796418492


### Assignment 4

In [49]:
for i in range(0,len(mtx_arr)):
    img = cv.imread('../calibration_frames/img_0005.png')
    h,  w = img.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx_arr[i], dist_arr[i], (w,h), 1, (w,h))
    # undistort
    dst = cv.undistort(img, mtx_arr[i], dist_arr[i], None, newcameramtx)
    # crop the image
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    cv.imwrite(f"calib{i}result.png", dst)

## Exercise 3

### Assignment 5

In [67]:
shadow = cv.imread("../afbeeldingen/shadow.png")
copy_shadow = shadow.copy()
rows, cols = shadow.shape[:2]

m = -0.15
horizontal_shear_M = np.array([[1, m, 0], [0, 1, 0]], dtype="float32")

print(horizontal_shear_M.dtype)

# move to the right
horizontal_shear_M[0, 2] += 260


shear_trans = cv.warpAffine(copy_shadow, horizontal_shear_M, (2*cols, rows))

print(shear_trans.dtype)
to_screen("original_shadow", shadow)
to_screen("shear_trans", shear_trans)


float32
uint8


### Assignment 6

In [3]:
shadow_box = cv.imread("../afbeeldingen/shadow_box.png")

# select the 4 corner points as source for getPerspectiveTransform starting from left bove corner and select other points counterwise
corner_points = []


def onMouse(k, x, y, s, param):
    if k == cv.EVENT_LBUTTONDOWN:
        param[0] += 1
        corner_points.append((x,y))
        # print(f"{x} and {y}")


# create a window
cv.namedWindow('klik')


aantal = [0]
# bind the callback function to window
cv.setMouseCallback('klik', onMouse, aantal)


cv.imshow('klik', shadow_box)
key = cv.waitKey()
while (key != 27):
    key = cv.waitKey()
cv.destroyAllWindows()



In [4]:

# select the 4 points as dst for getPerspectiveTransform with right upper corner such that the shape is rectangular 
target_points = corner_points[:3]

# width of bottom right point and height of top left point
target_points.append((corner_points[2][0], corner_points[0][1]))

# test result to see if rectangle
img = np.zeros((600,600,3), np.uint8)
img_copy = img.copy()
print(target_points)
for x, y in target_points:
    print(f"{x} and {y}")
    image = cv.circle(img_copy, (x,y), radius=0, color=(0, 0, 255), thickness=-1)

to_screen("img", img_copy)


    

[(109, 88), (117, 541), (299, 538), (299, 88)]
109 and 88
117 and 541
299 and 538
299 and 88


In [5]:
corner_np = np.float32(corner_points)
target_np = np.float32(target_points)


transform_mat = cv.getPerspectiveTransform(np.float32(corner_points), np.float32(target_points))
transform_mat

array([[ 4.63830727e+00,  7.28028440e-01, -3.95267441e+02],
       [ 9.68845187e-03,  5.21407288e+00, -3.19114998e+02],
       [ 1.10096044e-04,  6.67916000e-03,  1.00000000e+00]])

In [6]:

out = cv.warpPerspective(shadow_box,transform_mat,(shadow_box.shape[1], shadow_box.shape[0]))
to_screen("out", out)